In [13]:
from pathlib import Path
from PIL import Image

import albumentations as albu
from albumentations.augmentations.domain_adaptation import HistogramMatching
from albumentations.pytorch import ToTensorV2
import cv2
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import Dataset, DataLoader

In [8]:
data_path = Path("/media/imd/data/dataset/nlmk/machines/class_dataset_temp/")

In [9]:
list(data_path.iterdir())

[PosixPath('/media/imd/data/dataset/nlmk/machines/class_dataset_temp/test'),
 PosixPath('/media/imd/data/dataset/nlmk/machines/class_dataset_temp/train'),
 PosixPath('/media/imd/data/dataset/nlmk/machines/class_dataset_temp/valid')]

In [10]:
train_path = data_path / "train"
valid_path = data_path / "valid"
test_path = data_path / "test"

In [15]:
normalize_transform = albu.Compose(
    [
        albu.Normalize(),
        ToTensorV2()
    ]
)

train_pre_transform = HistogramMatching(
    [
        "/home/imd/Projects/nlmk-machines-ladle-status/img_hist_match/000014.png",
        "/home/imd/Projects/nlmk-machines-ladle-status/img_hist_match/000021.png",
        "/home/imd/Projects/nlmk-machines-ladle-status/img_hist_match/000027.png",
        "/home/imd/Projects/nlmk-machines-ladle-status/img_hist_match/1685436617.136851.png",
        "/home/imd/Projects/nlmk-machines-ladle-status/img_hist_match/1685533209.644287.png",
        "/home/imd/Projects/nlmk-machines-ladle-status/img_hist_match/1685555835.991394.png",
    ],
    blend_ratio=(0.5, 1.0),
    read_fn=lambda x: cv2.imread(x, -1),
    always_apply=False,
    p=0.7
)
train_augmentations = albu.Compose([
    albu.HorizontalFlip(p=0.5),
    albu.Rotate(limit=10, p=0.5),
    albu.Perspective(p=0.5, pad_mode=cv2.BORDER_REFLECT_101),
    albu.Resize(256, 256, p=1),
    *normalize_transform
])

valid_augmentations = albu.Compose([
    albu.Resize(256, 256, p=1),
    *normalize_transform
])

test_augmentations = albu.Compose([
    albu.Resize(256, 256, p=1),
    *normalize_transform
])

In [12]:
img = cv2.imread("/home/imd/Projects/nlmk-machines-ladle-status/img_hist_match/000014.png", -1)

In [26]:
class FrameDataset(Dataset):
    
    def __init__(self, file_path: Path, pre_aug=None, post_aug=None):
        self.files = list(set(file_path.rglob("*png")))
        self.pre_aug = pre_aug
        self.post_aug = post_aug
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, i):
        file_path = self.files[i]
        image = cv2.imread(str(file_path), cv2.IMREAD_UNCHANGED)
        # if image.shape[-1] != 3:
        #     image = cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
        # image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB) HW2
        if self.pre_aug is not None:
            image = self.pre_aug(image=image)["image"]
        image_normalized = cv2.normalize(image, None, 0, 255, cv2.NORM_MINMAX, dtype=cv2.CV_8U)
        color_frame = cv2.applyColorMap(image_normalized, cv2.COLORMAP_INFERNO)
        if self.post_aug is not None:
            color_frame = self.post_aug(image=color_frame)["image"]
        label = [
            (file_path.parent / "Flow" / file_path.name).is_file(),
            (file_path.parent / "Barrier" / file_path.name).is_file(),
            (file_path.parent / "Bubble" / file_path.name).is_file(),
        ]
        return color_frame, torch.FloatTensor(label)

In [27]:
train_dataset = FrameDataset(train_path, train_pre_transform, train_augmentations)
valid_dataset = FrameDataset(valid_path, None, valid_augmentations)
test_dataset = FrameDataset(test_path, None, test_augmentations)

In [30]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
valid_dataloader = DataLoader(valid_dataset, batch_size=32, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [31]:
train_dataset[0][0].shape

/home/imd/miniconda3/envs/mlisuct/lib/python3.8/site-packages/albumentations/augmentations/domain_adaptation.py:81: FutureWarning: `multichannel` is a deprecated argument name for `match_histograms`. It will be removed in version 1.0. Please use `channel_axis` instead.
  matched = match_histograms(np.squeeze(img), np.squeeze(reference_image), multichannel=True)


torch.Size([3, 256, 256])

In [33]:
next(iter(train_dataloader))[0].shape

torch.Size([32, 3, 256, 256])

In [34]:
from torchvision import models

In [36]:
model = models.resnet50(pretrained=True)

In [37]:
model

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [38]:
model.fc = nn.Linear(in_features=2048, out_features=3, bias=True)

In [41]:
optimizer = optim.Adam(model.parameters(), lr=1e-4)

In [42]:
sheduler = StepLR(optimizer, step_size=1000, gamma=0.5)

In [43]:
loss_f = nn.BCEWithLogitsLoss()